In [2]:
import pandas as pd 
import numpy as np
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [3]:
rs = 1

In [123]:
def read_data(file):
    """
    
    """
    df = (pd.read_csv(file, dtype = {
                            "PassengerId":np.int32,
                            "Name": "object",
                            "Pclass":"object",
                            "Survived":np.int32,
                            "Sex":"object",
                            "Age":np.float,
                            "SibSp":"object",
                            "Embarked":"object",
                            "Cabin":"object",
                            "Fare":np.float64,
                        },)
            .set_index("PassengerId")
        )
    return df
    

In [124]:
filename = "../data/train.csv"
data = read_data(filename)

In [129]:
data.Name.str#[0]

In [135]:
data.loc[1,:].Name

'Braund, Mr. Owen Harris'

In [101]:
data['age_cat'] = np.nan

In [140]:
def find_age_cat(x):
    """
    
    """
    if x.Name.lower().find('mr')!=-1:
        x.age_cat = 'older'
    elif x.Name.lower().find('mrs')!=-1:
        x.age_cat = 'older'
    elif x.Name.lower().find('master')!=-1:
        x.age_cat = 'younger'
    else: 
        x.age_cat = 'unknown'
    return x


In [167]:
def onehot_name(df):
    mrs = '|'.join(['Mrs\.','Mme\.', 'Ms\.', 'Dr\.', 'Lady\.', 'Countess\.'])    
    mr = '|'.join(['Mr\.','Don\.','Rev\.','Dr\.', 'Major\.', 'Sir\.', 'Col\.', 'Capt\.', 'Jonkheer\.'])
    miss = '|'.join(['Miss\.', 'Mlle\.'])
    return df.assign(Name = lambda x : x['Name'].str.replace('\(.*\)', '', regex=True),
                     Mr =  lambda x : x['Name'].str.contains(mr).astype(int) ,
                     Miss = lambda x : x['Name'].str.contains(miss).astype(int),
                     Mrs = lambda x : x['Name'].str.contains(mrs).astype(int),
                     Master = lambda x : (x['Name'].str.find('Master.') > -1).astype(int))

In [176]:
data = data.pipe(onehot_name)

In [127]:
data['age_cat'] = np.nan

In [141]:
data.apply(find_age_cat,axis=1).age_cat.str.contains('unknown').sum()

204

In [183]:
X_train = data.drop(['Survived','Name','Ticket','age_cat'],axis=1) 
y_train = data.Survived

In [184]:
rf = RandomForestClassifier()
quick_x = X_train.dropna(axis=0)

rf.fit(pd.get_dummies(quick_x),y_train.loc[quick_x.index])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [185]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

imp_mean = IterativeImputer(random_state=0, sample_posterior=True)

In [186]:
imp_mean.fit(X_train)

ValueError: could not convert string to float: 'male'

In [188]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
imp_mean = IterativeImputer(random_state=0)
imp_mean.fit([[np.nan, 2, 3], [4, np.nan, 6], [10, 5, 9]])


IterativeImputer(add_indicator=False, estimator=None,
                 imputation_order='ascending', initial_strategy='mean',
                 max_iter=10, max_value=None, min_value=None,
                 missing_values=nan, n_nearest_features=None, random_state=0,
                 sample_posterior=False, skip_complete=False, tol=0.001,
                 verbose=0)

In [200]:
# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['Age','Fare']
numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(random_state=0,initial_strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['Embarked', 'Sex', 'Pclass']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent',missing_values=np.nan)),
    ('onehot', OneHotEncoder(handle_unknown='error', drop="first"))
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numeric_features)
        ])

In [201]:
preprocessor.fit(X_train)

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('cat',
                                 Pipeline(memory=None,
                                          steps=[('imputer',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='most_frequent',
                                                                verbose=0)),
                                                 ('onehot',
                                                  OneHotEncoder(categories='auto',
                                                                drop='first',
                

In [195]:
ohe = OneHotEncoder(handle_unknown='error', drop="first")
imp = IterativeImputer(random_state=0)

In [197]:
imp.fit_transform(X_train)

ValueError: could not convert string to float: 'male'

In [196]:
ohe.fit_transform(X_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [194]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Mr,Miss,Mrs,Master
PassengerId,,,,,,,,,,,,
1,3,male,22.0,1,0,7.2500,NaN,S,1,0,0,0
2,1,female,38.0,1,0,71.2833,C85,C,0,0,1,0
3,3,female,26.0,0,0,7.9250,NaN,S,0,1,0,0
4,1,female,35.0,1,0,53.1000,C123,S,0,0,1,0
5,3,male,35.0,0,0,8.0500,NaN,S,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
887,2,male,27.0,0,0,13.0000,NaN,S,1,0,0,0
888,1,female,19.0,0,0,30.0000,B42,S,0,1,0,0
889,3,female,NaN,1,2,23.4500,NaN,S,0,1,0,0


In [193]:
preprocessor.fit(X_train)

ValueError: Input contains NaN

In [24]:
pipe_rfr = Pipeline([("pre_pros",preprocessor), 
                     ("rfr",RandomForestClassifier(n_estimators=10)) ])
params_rfr = {"rfr__n_estimators":[1,5,10,50,100,250,500,1000],
              "rfr__max_depth":[2,5,10,20,50,75,150]}

#cross validation for inner and outer loopn in nested cv
cv_outer = StratifiedKFold(shuffle=True, random_state = rs)
cv_inner = StratifiedKFold(shuffle=True, random_state = rs)

gs_rf = GridSearchCV(estimator = pipe_rfr,
                  param_grid = params_rfr,
                  cv = cv_inner,
                  scoring = "accuracy",
                  n_jobs=1)

In [25]:
gs_rf.fit(X_train,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('pre_pros',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('cat',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('imputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                      

In [71]:
import os

In [76]:
os.listdir("../data/test.csv")

NotADirectoryError: [Errno 20] Not a directory: '../data/test.csv'

In [77]:
pd.read_csv("../data/test.csv")

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
